<a href="https://colab.research.google.com/github/jukdang/AI_repo/blob/main/Dacon_toursism%20data%20classification/Union.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install gluonnlp
!pip install mxnet
!pip install sentencepiece
!pip install transformers
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install efficientnet_pytorch
!pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 4.7 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595726 sha256=d8d4b3a0713a3804fb357b0aaa427570ba73afbff61272c0ec1590abe19d546a
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 7.7 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=10a79a37b83a633cf6c933b9968fdd464046706b441b386fe7f3751aec83a051
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████▌                   | 834.1 MB 122.0 MB/s eta 0:00:11tcmalloc: large alloc 1147494400 bytes == 0x396a4000 @  0x7f577ae82615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



tcmalloc: large alloc 1434370048 bytes == 0x7dcfa000 @  0x7f577ae82615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |████████████████████            | 1336.2 MB 1.2 MB/s eta 0:11:32tcmalloc: large alloc 1792966656 bytes == 0x2b2c000 @  0x7f577ae82615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |█████████████████████████▎      | 1691.1 MB 1.2 MB/s eta 0:06:24tcmalloc: large alloc 2241208320 bytes == 0x6d914000 @  0x7f577ae82615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 

In [1]:
import gluonnlp as nlp

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#efficientNet
from efficientnet_pytorch import EfficientNet

In [2]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import torchvision
from torchvision import transforms

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, IterableDataset, DataLoader
from tqdm import tqdm, tqdm_notebook

import cv2
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd ./drive/MyDrive/

/content/drive/MyDrive


In [5]:
device = torch.device("cuda:0")

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

1
A100-SXM4-40GB
There are 1 GPU(s) available.
We will use the GPU: A100-SXM4-40GB


In [6]:
bertmodel, vocab = get_pytorch_kobert_model()
efficientnet = EfficientNet.from_pretrained('efficientnet-b4')

using cached model. /content/drive/MyDrive/.cache/kobert_v1.zip
using cached model. /content/drive/MyDrive/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
Loaded pretrained weights for efficientnet-b4


In [7]:
train_df = pd.read_csv('train_processed2.csv')
train_df['drive_img_path'] = './' + train_df['resized_img_path'].str[8:]
train_df

,id,resized_img_path,overview,sentence_length,category1,category2,category3,tokenized_overview,drive_img_path
0,TRAIN_00000,./image/resized_train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,217,0,0,0,"['소안항', '은', '조용한', '섬', '으로', '인근', '해안이', '청...",./resized_train/TRAIN_00000.jpg
1,TRAIN_00001,./image/resized_train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,244,1,1,1,"['경기도', '이천시', '모가면', '에', '있는', '골프장', '으로', ...",./resized_train/TRAIN_00001.jpg
2,TRAIN_00002,./image/resized_train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,160,2,2,2,"['금오산성', '숯불', '갈비', '는', '한우', '고기', '만을', '전...",./resized_train/TRAIN_00002.jpg
3,TRAIN_00003,./image/resized_train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,67,2,2,2,"['철판', '위', '에서', '요리', '하는', '안동찜닭', '을', '맛볼...",./resized_train/TRAIN_00003.jpg
4,TRAIN_00004,./image/resized_train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,63,2,2,2,"['※', '영업', '시간', '10:30', '~', '20:30', '\n\n...",./resized_train/TRAIN_00004.jpg
...,...,...,...,...,...,...,...,...,...
16981,TRAIN_16981,./image/resized_train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,50,2,2,2,"['해발', '12000', 'm', '에', '자리', '한', '식담', '겸'...",./resized_train/TRAIN_16981.jpg
16982,TRAIN_16982,./image/resized_train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,471,4,7,11,"['설악힐', '호텔', '은', '동해', '고속도로', '속초', '톨게이트',...",./resized_train/TRAIN_16982.jpg
16983,TRAIN_16983,./image/resized_train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,282,4,7,11,"['충남', '서산시', '중심', '가에', '위치', '한', '줌', '모텔'...",./resized_train/TRAIN_16983.jpg
16984,TRAIN_16984,./image/resized_train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,158,1,1,3,"['토', '토큰', '바위', '캠핑장', '은', '경기도', '가평', '지역...",./resized_train/TRAIN_16984.jpg


In [8]:
num_classes = len(train_df['category3'].unique())

In [9]:
train, test = train_test_split(train_df[['overview', 'drive_img_path', 'category3']], train_size=0.9, test_size=0.1)
train_df = train.to_numpy()
test_df = test.to_numpy()

In [ ]:
class InputIterableDataset(IterableDataset):
    def __init__(self, dataset, sentence_idx=0, img_idx=1, label_idx=2, bert_tokenizer=None, max_len=400, pad=True, pair=False):
        self.dataset = dataset
        self.transform_sentence = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.transform_image = transforms.Compose([transforms.Resize((224,224)), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def __iter__(self):
      for overview, img_path, label in self.dataset:
        sentence = self.transform_sentence(overview)
        img = self.transform_image(torchvision.io.read_image(img_path).type(torch.float32))
        label = np.int32(label)
        yield (sentence, img, label)

In [24]:
# Setting parameters 필수
max_len = 4096
batch_size = 16
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 1000
learning_rate =  5e-5

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/drive/MyDrive/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train = InputIterableDataset(train_df, 0, 1, 2, tok, max_len, True, False)
data_test = InputIterableDataset(test_df, 0, 1, 2, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  


In [ ]:
class Classifier(nn.Module):
    def __init__(self,
                 eff,
                 bert,
                 bert_hidden_size = 768,
                 eff_hidden_size = 1000,
                 num_classes = num_classes,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(Classifier, self).__init__()
        self.eff = eff
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.final = nn.Sequential(
            nn.Linear(bert_hidden_size + eff_hidden_size, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )
        self.classifier = nn.Linear(bert_hidden_size + eff_hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids, image):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, bert_out = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        eff_out = self.eff(image)

        if self.dr_rate:
            bert_out = self.dropout(bert_out)
            eff_out = self.dropout(eff_out)
        
        cat = torch.cat([bert_out, eff_out], dim=1)
        out = self.final(cat)
        #out = self.classifier(cat)

        return out

In [ ]:
model = Classifier(efficientnet, bertmodel,  dr_rate=0.4).to(device)

In [ ]:
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

t_total = len(train_df) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (sentence, img, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()

        token_ids, valid_length, segment_ids = sentence
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length.to(device)
        img = img.float().to(device)
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids, img)

        loss = loss_function(out, label)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))

        del img, label, out
        torch.cuda.empty_cache()

    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    
    model.eval()
    with torch.no_grad():
      for batch_id, (sentence, img, label) in enumerate(tqdm_notebook(test_dataloader)):
          token_ids, valid_length, segment_ids = sentence
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)
          valid_length= valid_length.to(device)
          img = img.float().to(device)
          label = label.long().to(device)

          out = model(token_ids, valid_length, segment_ids, img)
          
          test_acc += calc_accuracy(out, label)

          del img, label, out
          torch.cuda.empty_cache()
          
      print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  


epoch 1 batch id 1 loss 4.861398696899414 train acc 0.0
epoch 1 batch id 1001 loss 3.242445945739746 train acc 0.14017232767232768
epoch 1 batch id 2001 loss 3.280475378036499 train acc 0.2172663668165917
epoch 1 batch id 3001 loss 2.958049774169922 train acc 0.26105881372875706
epoch 1 batch id 4001 loss 2.6562726497650146 train acc 0.2954730067483129
epoch 1 train acc 0.3146835774058577


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 batch id 1 loss 0.845151960849762 train acc 0.75
epoch 4 batch id 1001 loss 0.5024996399879456 train acc 0.8099400599400599
epoch 4 batch id 2001 loss 0.9467456936836243 train acc 0.8170914542728636
epoch 4 batch id 3001 loss 0.3323690593242645 train acc 0.8325974675108297
epoch 4 batch id 4001 loss 0.11110786348581314 train acc 0.8455386153461635
epoch 4 train acc 0.8538702928870293


0it [00:00, ?it/s]

epoch 4 test acc 0.4131619937694705


0it [00:00, ?it/s]

epoch 5 batch id 1 loss 1.4527984857559204 train acc 0.875
epoch 5 batch id 1001 loss 0.22080686688423157 train acc 0.904532967032967
epoch 5 batch id 2001 loss 0.20801971852779388 train acc 0.915635932033983
epoch 5 batch id 3001 loss 0.12436983734369278 train acc 0.9227757414195268
epoch 5 batch id 4001 loss 0.003632171545177698 train acc 0.9293614096475881


2번째

In [10]:
class InputIterableDataset(IterableDataset):
    def __init__(self, dataset, bert_tokenizer=None, max_len=400, pad=True, pair=False):
        self.dataset = dataset
        self.transform_sentence = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

    def __iter__(self):
      for overview, img_path, label in self.dataset:
        sentence = self.transform_sentence(overview)
        label = np.int32(label)
        yield (sentence, label)

In [11]:
# Setting parameters 필수
max_len = 512
batch_size = 32
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 1000
learning_rate =  5e-5

In [12]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = InputIterableDataset(train_df, tok, max_len, True, False)
data_test = InputIterableDataset(test_df, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

using cached model. /content/drive/MyDrive/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [58]:
tmp = next(iter(train_dataloader))
print(tmp[0][0].shape, tmp[0][1].shape, tmp[0][2].shape)

torch.Size([32, 512]) torch.Size([32]) torch.Size([32, 512])


In [13]:
class Classifier(nn.Module):
    def __init__(self,
                 bert,
                 bert_hidden_size = 768,
                 num_classes = num_classes,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(Classifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.final = nn.Sequential(
            nn.Linear(bert_hidden_size, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
        self.classifier = nn.Linear(bert_hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, bert_out = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))

        if self.dr_rate:
            bert_out = self.dropout(bert_out)
        
        #cat = torch.cat([bert_out, eff_out], dim=1)
        #out = self.final(cat)
        out = self.classifier(bert_out)

        return out

In [14]:
model3 = Classifier(bertmodel, dr_rate=0.4).to(device)

In [15]:
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model3.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model3.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

t_total = len(train_df) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model3.train()
    for batch_id, (sentence, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()

        token_ids, valid_length, segment_ids = sentence
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length.to(device)
        label = label.long().to(device)

        out = model3(token_ids, valid_length, segment_ids)

        loss = loss_function(out, label)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model3.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))

        del label, out
        torch.cuda.empty_cache()

    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    
    model3.eval()
    with torch.no_grad():
      for batch_id, (sentence, label) in enumerate(tqdm_notebook(test_dataloader)):
          token_ids, valid_length, segment_ids = sentence
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)
          valid_length= valid_length.to(device)
          label = label.long().to(device)

          out = model3(token_ids, valid_length, segment_ids)
          
          test_acc += calc_accuracy(out, label)

          del label, out
          torch.cuda.empty_cache()
          
      print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

epoch 1 batch id 1 loss 5.003971099853516 train acc 0.0
epoch 1 batch id 1001 loss 4.083083152770996 train acc 0.14763361638361638
epoch 1 batch id 2001 loss 3.993147134780884 train acc 0.174444027986007
epoch 1 train acc 0.17969176368928505


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

epoch 9 test acc 0.21913580246913586


0it [00:00, ?it/s]

epoch 10 batch id 1 loss 3.3091049194335938 train acc 0.28125
epoch 10 batch id 1001 loss 3.770905017852783 train acc 0.20192307692307693


KeyboardInterrupt: ignored

3번째

In [21]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 68.0 MB/s 
     |████████████████████████████████| 163 kB 82.8 MB/s 


In [22]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=4096)

In [10]:
train_df = pd.read_csv('train_processed2.csv')
train_df['drive_img_path'] = './' + train_df['resized_img_path'].str[8:]

In [11]:
train, test = train_test_split(train_df, train_size=0.9, test_size=0.1, shuffle=True)

In [12]:
train_vectors = vectorizer.fit_transform(train['overview'])
train_vectors = train_vectors.todense()

val_vectors = vectorizer.transform(test['overview'])
val_vectors = val_vectors.todense()

In [13]:
train = train[['drive_img_path', 'category3']]
test = test[['drive_img_path', 'category3']]
train_df = train.to_numpy()
test_df = test.to_numpy()

In [25]:
# Setting parameters 필수
max_len = 4096
batch_size = 64
warmup_ratio = 0.1
num_epochs = 15
max_grad_norm = 1
log_interval = 1000
learning_rate =  5e-5

In [14]:
class InputIterableDataset(IterableDataset):
    def __init__(self, dataset, vectors):
        self.dataset = dataset
        self.vectors = vectors
        self.transform_image = A.Compose([A.Resize(128,128),
                                          A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),\
                                          ToTensorV2()])

    def __iter__(self):
      for (img_path, label), overview in zip(self.dataset, self.vectors):
        sentence = torch.tensor(overview).squeeze(0)

        img = cv2.imread(img_path)
        img = self.transform_image(image=img)['image']

        label = np.int32(label)
        yield (sentence, img, label)

In [15]:
data_train = InputIterableDataset(train_df, train_vectors)
data_test = InputIterableDataset(test_df, val_vectors)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
tmp = next(iter(train_dataloader))

In [26]:
class CustomModel(nn.Module):
    def __init__(self, num_classes=num_classes):
        super(CustomModel, self).__init__()
        # Image
        self.cnn_extract = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1),
                # input_channel = 3 : RGB 3개의 채널이기 때문
                # out_channel = 8 : 출력하는 채널 8개
                # stride = 1 : stride 만큼 이미지 이동하면서 cnn 수행
                # padding = 1 : zero-padding할 사이즈
            nn.ReLU(), # 사용할 활성화 함수: Relu를 사용
            nn.MaxPool2d(kernel_size=2, stride=2), # 최댓값을 뽑아내는 맥스 풀링
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=4, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # Text
        self.nlp_extract = nn.Sequential(
            nn.Linear(4096, 2048), # 선형회귀. 4096개의 입력으로 2048개의 출력
            nn.ReLU(),
            nn.Linear(2048, 1024), # 선형회귀. 2048개의 입력으로 1024개의 출력
        )
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(4160, num_classes)
            # 선형회귀. 4160개의 입력으로 num_classes, 즉 cat3의 종류 개수만큼의 출력
            # 근데 왜 4160개? "4160 - 1024 = 3136"이고 "3136 / 64 = 49". 즉 이미지는 "7*7*64"로 출력됨.
        )
            

    def forward(self, text, img):
        img_feature = self.cnn_extract(img) # cnn_extract 적용
        img_feature = torch.flatten(img_feature, start_dim=1) # 1차원으로 변환
        text_feature = self.nlp_extract(text) # nlp_extract 적용
        feature = torch.cat([img_feature, text_feature], axis=1) # 2개 연결(3136 + 1024)
        output = self.classifier(feature) # classifier 적용
        return output

In [27]:
model2 = CustomModel().to(device)

In [28]:
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model2.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model2.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

t_total = len(train_df) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model2.train()
    for batch_id, (sentence, img, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()

        sentence = sentence.float().to(device)
        img = img.float().to(device)
        label = label.long().to(device)

        out = model2(sentence, img)

        loss = loss_function(out, label)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model2.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))

        del img, label, out
        torch.cuda.empty_cache()

    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    
    model2.eval()
    with torch.no_grad():
      for batch_id, (sentence, img, label) in enumerate(tqdm_notebook(test_dataloader)):
          sentence = sentence.float().to(device)
          img = img.float().to(device)
          label = label.long().to(device)

          out = model2(sentence, img)
          
          test_acc += calc_accuracy(out, label)

          del img, label, out
          torch.cuda.empty_cache()
          
      print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

epoch 1 batch id 1 loss 4.865333557128906 train acc 0.015625
epoch 1 batch id 1001 loss 4.492317199707031 train acc 0.08897352647352648
epoch 1 train acc 0.10806984594903007


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

...

epoch 14 test acc 0.6079695767195766


0it [00:00, ?it/s]

epoch 15 batch id 1 loss 0.056838419288396835 train acc 0.984375
epoch 15 batch id 1001 loss 0.1306941658258438 train acc 0.9958322927072927
epoch 15 train acc 0.9963389121338913


0it [00:00, ?it/s]

epoch 15 test acc 0.6056547619047619
